In [1]:
import os, json, numpy as np, pandas as pd, torch, torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
from torchvision.models import ResNet18_Weights
from PIL import Image
import joblib
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt

print("PyTorch:", torch.__version__)
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {DEVICE}")


PyTorch: 2.9.1+cu128
Device: cpu


In [2]:
PROCESSED_DIR = "data/processed"
TRAIN_IMG_DIR = "data/images/train"
TEST_IMG_DIR = "data/images/test"

# Load tabular arrays
X_train_tab = np.load(f"{PROCESSED_DIR}/X_train_tabular.npy")
X_val_tab = np.load(f"{PROCESSED_DIR}/X_val_tabular.npy")
y_train = np.load(f"{PROCESSED_DIR}/y_train_log_price.npy")
y_val = np.load(f"{PROCESSED_DIR}/y_val_log_price.npy")
train_ids = np.load(f"{PROCESSED_DIR}/train_ids.npy")
val_ids = np.load(f"{PROCESSED_DIR}/val_ids.npy")

scaler = joblib.load(f"{PROCESSED_DIR}/tabular_scaler.joblib")
with open(f"{PROCESSED_DIR}/metadata.json") as f:
    meta = json.load(f)

print(f"✅ Tabular: Train={X_train_tab.shape}, Val={X_val_tab.shape}")
print(f"✅ Images ready: {TRAIN_IMG_DIR}, {TEST_IMG_DIR}")


✅ Tabular: Train=(12901, 21), Val=(3226, 21)
✅ Images ready: data/images/train, data/images/test


In [7]:
# EXTRACT IMAGE FEATURES FROM FOLDER
import torch
from torchvision import models, transforms
from torch.utils.data import DataLoader
from PIL import Image
import numpy as np
import pandas as pd
from tqdm import tqdm
import os
import joblib

# Feature extractor (ResNet18 → 512 features)
device = "cuda" if torch.cuda.is_available() else "cpu"
backbone = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
backbone.fc = torch.nn.Identity()  # Remove classifier → 512 features
backbone = backbone.to(device).eval()

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406], [0.229,0.224,0.225])
])

def extract_features_from_folder(img_dir, ids):
    """Extract 512 features per image"""
    features = {}
    
    for prop_id in tqdm(ids, desc=f"Extracting {img_dir}"):
        img_path = os.path.join(img_dir, f"{int(prop_id)}.png")
        
        if os.path.exists(img_path):
            img = Image.open(img_path).convert('RGB')
            img_t = transform(img).unsqueeze(0).to(device)
            with torch.no_grad():
                feat = backbone(img_t).cpu().numpy().flatten()
            features[str(prop_id)] = feat
        else:
            # Zero features for missing images
            features[str(prop_id)] = np.zeros(512)
    
    return features

print("🔄 Extracting train image features...")
train_img_features = extract_features_from_folder(TRAIN_IMG_DIR, train_ids)

print("🔄 Extracting val image features...")
val_img_features = extract_features_from_folder(TRAIN_IMG_DIR, val_ids)

# Save
joblib.dump(train_img_features, f"{PROCESSED_DIR}/train_img_features.pkl")
joblib.dump(val_img_features, f"{PROCESSED_DIR}/val_img_features.pkl")
print("✅ Image features saved!")



🔄 Extracting train image features...


Extracting data/images/train: 100%|█| 12901/12901 [05:09<00:00, 41.70


🔄 Extracting val image features...


Extracting data/images/train: 100%|█| 3226/3226 [01:13<00:00, 43.81it


✅ Image features saved!


In [ ]:
# 🔍 VERIFY IMAGE FEATURES BEFORE COMBINING
print("🔍 Verifying extracted image features...")

# Load your extracted features
train_img_feat = joblib.load(f"{PROCESSED_DIR}/train_img_features.pkl")
val_img_feat = joblib.load(f"{PROCESSED_DIR}/val_img_features.pkl")

print(f"Train images extracted: {len(train_img_feat)}")
print(f"Val images extracted:   {len(val_img_feat)}")

# Check sample shapes
sample_train_id = list(train_img_feat.keys())[0]
sample_val_id = list(val_img_feat.keys())[0]

print(f"\n Sample train ID {sample_train_id}: {train_img_feat[sample_train_id].shape}")
print(f" Sample val ID   {sample_val_id}:   {val_img_feat[sample_val_id].shape}")

# Verify ALL are 512
train_shapes = set(len(feat) for feat in train_img_feat.values())
val_shapes = set(len(feat) for feat in val_img_feat.values())
print(f"\n Train feature sizes: {train_shapes}")  # {512}
print(f"Val feature sizes:   {val_shapes}")    # {512}

assert 512 in train_shapes and len(train_shapes) == 1, "Train features not uniform 512!"
assert 512 in val_shapes and len(val_shapes) == 1, "Val features not uniform 512!"
print("ALL FEATURES = 512 ✓")

# Quick DataFrame preview (first 5 images)
train_sample_df = pd.DataFrame.from_dict(
    {k: v[:5] for k, v in list(train_img_feat.items())[:5]},  # first 5 features
    orient="index"
)
train_sample_df.columns = [f"img_feat_{i}" for i in range(5)]
print("\n Sample train features (first 5):")
print(train_sample_df.head())


In [ ]:
# 🟢 VERIFICATION PASSED → COMBINE FEATURES
print("\n Combining tabular + 512 image features...")

# Convert dict to arrays (match ID order)
train_img_array = np.array([train_img_feat[str(int(id))] for id in train_ids])
val_img_array = np.array([val_img_feat[str(int(id))] for id in val_ids])

# Stack: tabular + images
X_train_full = np.hstack([X_train_tab, train_img_array])
X_val_full = np.hstack([X_val_tab, val_img_array])

print(f"✅ COMBINED FEATURES:")
print(f"   Train shape: {X_train_full.shape} (tabular + 512 img)")
print(f"   Val shape:   {X_val_full.shape}")
print(f"   New feature count: {X_train_full.shape[1]}")

# Save combined
np.save(f"{PROCESSED_DIR}/X_train_full.npy", X_train_full)
np.save(f"{PROCESSED_DIR}/X_val_full.npy", X_val_full)
print(" Saved full features!")


In [12]:
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score



# Ultra-fast histogram version
fusion_model = HistGradientBoostingRegressor(
    max_iter=500,           # 500 trees (vs 1000)
    max_depth=10,           # Slightly deeper
    learning_rate=0.08,
    random_state=42,
    verbose=1               # Progress bar
)

print("Training fusion model...")
fusion_model.fit(X_train_full, y_train)
print(" Training complete!")

# Results
y_val_pred_log = fusion_model.predict(X_val_full)
rmse = np.sqrt(mean_squared_error(np.expm1(y_val), np.expm1(y_val_pred_log)))
r2 = r2_score(np.expm1(y_val), np.expm1(y_val_pred_log))

print(f"\n FUSION RESULTS:")
print(f"   Tabular RMSE: 101496")
print(f"   Fusion RMSE:  {rmse:.0f}")
print(f"   Fusion R²:    {r2:.3f}")
print(f"   Improvement: {(101496-rmse)/101496*100:.1f}%")

# Save
import joblib
joblib.dump(fusion_model, f"{PROCESSED_DIR}/fusion_model.joblib")
print("Model saved!")


🚀 FAST HISTOGRADIENTBOOSTING (2 min)
Training fusion model...
Binning 0.050 GB of training data: 0.301 s
Binning 0.006 GB of validation data: 0.012 s
Fitting gradient boosted rounds:
Fit 110 trees in 3.162 s, (3410 total leaves)
Time spent computing histograms: 1.568s
Time spent finding best splits:  1.009s
Time spent applying splits:      0.080s
Time spent predicting:           0.006s
✅ Training complete!

🎯 FUSION RESULTS:
   Tabular RMSE: 101496
   Fusion RMSE:  101420
   Fusion R²:    0.888
   Improvement: 0.1%
✅ Model saved!


In [16]:
# 🧪 MISSING TEST FEATURES → CREATE NOW
print("🔄 Creating test_img_features.pkl...")

test_df = pd.read_csv("data/test.csv")
test_ids = test_df["id"].values

# Use your existing backbone/transform
test_img_features = extract_features_from_folder(TEST_IMG_DIR, test_ids)

joblib.dump(test_img_features, f"{PROCESSED_DIR}/test_img_features.pkl")
print(f"✅ Test features saved: {len(test_img_features)} images")


🔄 Creating test_img_features.pkl...


Extracting data/images/test: 100%|█| 5404/5404 [03:40<00:00, 24.47it/


✅ Test features saved: 5396 images


In [23]:
# 🏆 CORRECT FINAL SUBMISSION (creates all 21 features)
import joblib, numpy as np, pandas as pd, json

PROCESSED_DIR = "data/processed"

# Load
fusion_model = joblib.load(PROCESSED_DIR + "/fusion_model.joblib")
scaler = joblib.load(PROCESSED_DIR + "/tabular_scaler.joblib")
with open(PROCESSED_DIR + "/metadata.json", "r") as f:
    meta = json.load(f)
test_img_features = joblib.load(PROCESSED_DIR + "/test_img_features.pkl")
test_df = pd.read_csv("data/test.csv")

# 🔥 CREATE ALL 21 EXPECTED FEATURES
expected_features = meta["numeric_features"]  # 21 features


# Create DataFrame with ALL expected columns (missing = 0)
test_features_df = pd.DataFrame(0, index=test_df.index, columns=expected_features)
test_features_df["id"] = test_df["id"]  # Keep ID

# Fill available features from test.csv
for col in expected_features:
    if col in test_df.columns:
        test_features_df[col] = test_df[col].fillna(0)



# Transform ALL 21 features
test_tabular = scaler.transform(test_features_df[expected_features].values)


# Image features
test_img_array = np.array([test_img_features.get(str(int(id)), np.zeros(512)) for id in test_df["id"]])

# Combine & predict
X_test = np.column_stack([test_tabular, test_img_array])
test_prices = np.expm1(fusion_model.predict(X_test))

# Save
pd.DataFrame({
    "id": test_df["id"].astype(int),
    "predicted_price": test_prices
}).to_csv("submission.csv", index=False)

print("\n submission.csv SAVED!")
print(f"Samples: {len(test_prices)}")
print(f"Prices:  ${test_prices.min():.0f} - ${test_prices.max():.0f}")




 submission.csv SAVED!
Samples: 5404
Prices:  $131299 - $2932170
